## Importaciones

In [ ]:
# Importamos las bibliotecas que usaremos
from sklearn.feature_extraction.text import CountVectorizer  # Para crear bag-of-words
from sklearn.decomposition import LatentDirichletAllocation  # Para el modelo LDA
import numpy as np  # Para operaciones numéricas
import pandas as pd  # Para manipular datos tabulares
import json, re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suprimimos warnings para mayor claridad

## Carga de datos

In [29]:
# Lista de textos a procesar
with open('../DataSets/Noticias.json', encoding="utf8") as json_file:
    datos = json.load(json_file)

tuplas = list(zip([noticia.get("Title") for noticia in datos],[noticia.get("TextContent") for noticia in datos]))
df = pd.DataFrame(tuplas, columns =['Titular', 'Noticia'])
df   

,Titular,Noticia
0,Suspendido el partido Villarreal-Espanyol por ...,El temporal de lluvia y nieve afecta a áreas d...
1,Reino Unido y otros países aliados de Ucrania ...,Los países europeos de la OTAN y Canadá han de...
2,Los premios Oscar dan la gloria al cine indie ...,¿Qué premia exactamente Hollywood y su industr...
3,"Emilia Pérez, Karla Sofía Gascón, Demi Moore y...","Fue Beckett el que, en un arrebato no precisam..."
4,La Aemet retira también el aviso rojo por fuer...,La Agencia Estatal de Meteorología (Aemet) ha ...
5,España propone financiar la defensa de los paí...,España considera que la seguridad es un “Bien ...
6,El tequila lubricó la gran noche en la que Hol...,La bebida fluyó con abundancia en los cuatro n...
7,Trump congela toda la ayuda militar a Ucrania ...,"El presidente estadounidense, Donald Trump, ha..."
8,La Comunidad Valenciana y otras cinco regiones...,Este martes seguirá arreciando en el este de l...
9,Objetivo: acabar con la pesca fantasma y las r...,Dos biólogos de la Universitat de València par...


## Procesamiento inicial

In [30]:
# Lista de textos a procesar
documents = df['Noticia'].tolist()

# Configurar y crear el vectorizador
tf_vectorizer = CountVectorizer(
    stop_words=[],  # No eliminamos stopwords por ahora []
    min_df=1,      # Incluir palabras que aparecen al menos 1 vez
    max_df=1.0,    # Sin límite superior de frecuencia
    lowercase=True, # Convertir todo a minúsculas
    max_features=50000,  # Máximo número de palabras a considerar
    token_pattern='[a-zA-Z0-9]{3,}',  # Palabras de 3+ caracteres
    analyzer = 'word'
)

# Crear la matriz de documentos-términos
bag_of_words = tf_vectorizer.fit_transform(documents)

# Obtener el vocabulario
dictionary = tf_vectorizer.get_feature_names_out()
vocabulary = tf_vectorizer.vocabulary_

print("Estadísticas del preprocesamiento:")
print(f"- Tamaño del vocabulario: {len(dictionary)} palabras únicas")
print(f"- Dimensiones de la matriz: {bag_of_words.shape}")

# Mostrar las palabras más frecuentes
word_freq = bag_of_words.sum(axis=0).A1
top_words_idx = word_freq.argsort()[-10:][::-1]
print("\nPalabras más frecuentes:")
for idx in top_words_idx:
    print(f"- {dictionary[idx]}: {word_freq[idx]} apariciones")

Estadísticas del preprocesamiento:
- Tamaño del vocabulario: 390 palabras únicas
- Dimensiones de la matriz: (12, 390)

Palabras más frecuentes:
- que: 15 apariciones
- los: 12 apariciones
- para: 9 apariciones
- del: 9 apariciones
- las: 8 apariciones
- por: 8 apariciones
- una: 8 apariciones
- con: 7 apariciones
- han: 6 apariciones
- este: 5 apariciones


## Entrenamiento LDA

El algoritmo LDA tiene varios hiperparámetros importantes:

* n_topics: Número de tópicos a encontrar
    - Debe elegirse según el conocimiento del dominio
    - Se puede optimizar usando métricas como coherencia o perplejidad

* alpha: Prior de la distribución documentos-tópicos
    - alpha < 1: documentos se concentran en pocos tópicos
    - alpha > 1: documentos mezclan varios tópicos
    - alpha = 1: distribución uniforme

* beta: Prior de la distribución tópicos-palabras
    - beta < 1: tópicos más específicos (pocas palabras)
    - beta > 1: tópicos más generales (muchas palabras)
    - beta = 1: distribución uniforme





In [31]:
# Parámetros del modelo
n_topics = 2    # Número moderado de tópicos para empezar
alpha = 1.0     # Documentos algo especializados
beta = 0.1     # Tópicos bastante específicos

# Crear y entrenar el modelo
print("Configuración del modelo LDA:")
print(f"- Número de tópicos: {n_topics}")
print(f"- Alpha: {alpha}")
print(f"- Beta: {beta}")
print("\nIniciando entrenamiento...\n")

lda = LatentDirichletAllocation(
    n_components=n_topics,      # Número de tópicos
    doc_topic_prior=alpha,      # Prior documentos-tópicos
    topic_word_prior=beta,      # Prior tópicos-palabras
    max_iter=25,               # Máximo de iteraciones
    learning_method='online',   # Método de aprendizaje
    evaluate_every=1,          # Evaluar en cada iteración
    n_jobs=-1,                # Usar todos los cores
    random_state=0,           # Semilla para reproducibilidad
    verbose=1                 # Mostrar progreso
)

# Entrenar el modelo
lda.fit(bag_of_words)

Configuración del modelo LDA:
- Número de tópicos: 2
- Alpha: 1.0
- Beta: 0.1

Iniciando entrenamiento...

iteration: 1 of max_iter: 25, perplexity: 2758.9505
iteration: 2 of max_iter: 25, perplexity: 2360.5343
iteration: 3 of max_iter: 25, perplexity: 2063.0509
iteration: 4 of max_iter: 25, perplexity: 1844.5225
iteration: 5 of max_iter: 25, perplexity: 1679.2094
iteration: 6 of max_iter: 25, perplexity: 1550.4091
iteration: 7 of max_iter: 25, perplexity: 1447.6977
iteration: 8 of max_iter: 25, perplexity: 1364.3317
iteration: 9 of max_iter: 25, perplexity: 1295.7519
iteration: 10 of max_iter: 25, perplexity: 1238.7505
iteration: 11 of max_iter: 25, perplexity: 1190.9913
iteration: 12 of max_iter: 25, perplexity: 1150.7229
iteration: 13 of max_iter: 25, perplexity: 1116.5991
iteration: 14 of max_iter: 25, perplexity: 1087.5641
iteration: 15 of max_iter: 25, perplexity: 1062.7752
iteration: 16 of max_iter: 25, perplexity: 1041.5504
iteration: 17 of max_iter: 25, perplexity: 1023.3309
i

LatentDirichletAllocation(doc_topic_prior=1.0, evaluate_every=1,
                          learning_method='online', max_iter=25, n_components=2,
                          n_jobs=-1, random_state=0, topic_word_prior=0.1,
                          verbose=1)

## Análisis de Resultados

Se analizan los resultados de tres formas diferentes:

* Palabras más relevantes por tópico
* Documentos más representativos de cada tópico
* Distribución de tópicos en documentos específicos



In [32]:
# Configuración de visualización
no_top_words = 10      # Número de palabras top por tópico
no_top_documents = 2   # Número de documentos top por tópico

# Obtener las distribuciones
doc_topics = lda.transform(bag_of_words)  # Distribución de tópicos por documento
topics = lda.components_                  # Distribución de palabras por tópico


# 1. Detallar tópicos encontrados
print("TÓPICOS DESCUBIERTOS")
print("Cada tópico se representa por sus palabras más probables\n")

for topic_idx, topic in enumerate(topics):
    print(f" Tópico {topic_idx + 1}:")
    # Obtener índices de las palabras más probables
    top_words_idx = topic.argsort()[:-no_top_words-1:-1]
    top_words = [dictionary[i] for i in top_words_idx]
    top_probs = [topic[i] for i in top_words_idx]

    # Mostrar palabras y sus probabilidades
   
    for word, prob in zip(top_words, top_probs):
        print(f"   {word}: {prob:.4f}")
    print()



TÓPICOS DESCUBIERTOS
Cada tópico se representa por sus palabras más probables

 Tópico 1:
   que: 7.7819
   por: 5.1886
   los: 5.0779
   han: 5.0075
   las: 4.0323
   horas: 3.5349
   para: 3.0124
   rojo: 3.0002
   meteorolog: 2.4794
   provincia: 2.4732

 Tópico 2:
   del: 7.8141
   que: 6.8645
   los: 6.7024
   una: 6.0122
   para: 5.8944
   con: 4.9903
   este: 4.8988
   las: 3.9034
   ses: 3.0481
   verdad: 2.9999



In [33]:
# 2. Documentos más representativos por tópico
print("\n DOCUMENTOS MÁS REPRESENTATIVOS POR TÓPICO")
print("Se muestran los documentos que más peso tienen en cada tópico\n")

for topic_idx in range(n_topics):
    print(f" Tópico {topic_idx + 1}:")
    # Obtener los documentos más representativos
    top_doc_indices = np.argsort(doc_topics[:,topic_idx])[::-1][:no_top_documents]

    for doc_idx in top_doc_indices:
        title = df.iloc[doc_idx]['Titular']
        weight = doc_topics[doc_idx, topic_idx]
        print(f"   '{title}'")
        print(f"      Peso: {weight:.4f}")
    print()


 DOCUMENTOS MÁS REPRESENTATIVOS POR TÓPICO
Se muestran los documentos que más peso tienen en cada tópico

 Tópico 1:
   'Suspendido el partido Villarreal-Espanyol por la emergencia meteorológica.'
      Peso: 0.9846
   'La Aemet retira también el aviso rojo por fuertes lluvias en Castellón, que se mantiene en naranja.'
      Peso: 0.9734

 Tópico 2:
   'Los premios Oscar dan la gloria al cine indie y castigan una vez más a Netflix'
      Peso: 0.9871
   'Reino Unido y otros países aliados de Ucrania se comprometen a rearmar a Zelenski: 'Botas en el terreno y aviones en los cielos''
      Peso: 0.9814



In [34]:
# Para mostrar matriz documento-tópico
from IPython.display import display, HTML
import pandas as pd
#pd.set_option('display.max_columns', None)

topicnames = ["topic"+ str(x) for x in range(0, lda.n_components)]
norm_doc_topics = []
for i in doc_topics:
  norm_doc_topics.append([ "{0:.3f}".format(weight) for weight in i])

df = pd.DataFrame(norm_doc_topics,
                  columns=topicnames,
                  index=df['Titular'].tolist())

df

,topic0,topic1
Suspendido el partido Villarreal-Espanyol por la emergencia meteorológica.,0.985,0.015
Reino Unido y otros países aliados de Ucrania se comprometen a rearmar a Zelenski: 'Botas en el terreno y aviones en los cielos',0.019,0.981
Los premios Oscar dan la gloria al cine indie y castigan una vez más a Netflix,0.013,0.987
"Emilia Pérez, Karla Sofía Gascón, Demi Moore y la decencia, a la cabeza de los perdedores de la noche",0.022,0.978
"La Aemet retira también el aviso rojo por fuertes lluvias en Castellón, que se mantiene en naranja.",0.973,0.027
España propone financiar la defensa de los países de la UE con fondos europeos,0.965,0.035
El tequila lubricó la gran noche en la que Hollywood no quiso hablar de Donald Trump,0.027,0.973
Trump congela toda la ayuda militar a Ucrania para castigar y doblegar a Zelenski,0.961,0.039
"La Comunidad Valenciana y otras cinco regiones, bajo aviso por lluvia",0.186,0.814
Objetivo: acabar con la pesca fantasma y las redes abandonadas que matan y mutilan tortugas en el Mediterráneo,0.056,0.944


In [35]:
# Matriz tópico-palabra

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis])

# Assign Column and Index
df_topic_keywords.columns = dictionary
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,000,100,106,143,180,2017,2024,500,abundancia,academia,...,vieron,viles,volod,voluntad,willem,ximo,xito,zelenski,zonas,zorrilla
topic0,0.000579,0.000582,0.004247,0.004189,0.004206,0.000553,0.000561,0.004209,0.000569,0.000552,...,0.004200,0.004228,0.004178,0.004127,0.000543,0.004255,0.000539,0.004146,0.000572,0.000549
topic1,0.002948,0.002939,0.000371,0.000400,0.000396,0.002947,0.002938,0.000393,0.002929,0.002944,...,0.000376,0.000394,0.000394,0.000417,0.002958,0.000384,0.002931,0.000418,0.002920,0.002937


## Evaluación del modelo

Utilizamos dos métricas principales:

1. Log Likelihood (mayor es mejor):
   * Indica cómo de bien el modelo explica los datos
   * Valores más altos indican mejor ajuste

2. Perplejidad (menor es mejor):
   * Mide qué tan "sorprendido" está el modelo por los datos
   * Valores más bajos indican mejor generalización



In [36]:
# Calcular métricas
log_likelihood = lda.score(bag_of_words)
perplexity = lda.perplexity(bag_of_words)

print("MÉTRICAS DE EVALUACIÓN")
print(f"- Log Likelihood: {log_likelihood:.2f}")
print(f"- Perplejidad: {perplexity:.2f}")

# Comparar con diferentes valores de hiperparámetros
print("\nCOMPARACIÓN DE HIPERPARÁMETROS")
print("Probando diferentes configuraciones para encontrar el mejor modelo...")

# Probar diferentes números de tópicos
n_topics_range = [2,4,6,8]
results = []

for n_top in n_topics_range:
    model = LatentDirichletAllocation(
        n_components=n_top,
        doc_topic_prior=alpha,
        topic_word_prior=beta,
        max_iter=25,
        random_state=0
    )
    model.fit(bag_of_words)

    results.append({
        'n_topics': n_top,
        'perplexity': model.perplexity(bag_of_words),
        'log_likelihood': model.score(bag_of_words)
    })

# Mostrar resultados
results_df = pd.DataFrame(results)
print("\nResultados con diferentes números de tópicos:")
print(results_df)

MÉTRICAS DE EVALUACIÓN
- Log Likelihood: -3732.99
- Perplejidad: 943.43

COMPARACIÓN DE HIPERPARÁMETROS
Probando diferentes configuraciones para encontrar el mejor modelo...

Resultados con diferentes números de tópicos:
   n_topics  perplexity  log_likelihood
0         2  878.213841    -3693.950115
1         4  801.510384    -3644.141370
2         6  775.912329    -3626.451547
3         8  865.176829    -3685.798982
